In [0]:
s3_bucket = 'singaretti-mapreduce-test'
aws_access_key_id=''
aws_secret_access_key=''

In [0]:
def discover_document(s3_bucket):
  """
  This function access s3 bucket to discover document files to be processed
  :param s3_bucket: bucket to discover documents to be read
  """
  import boto3
  s3_resource = boto3.resource('s3',aws_access_key_id=aws_access_key_id,aws_secret_access_key=aws_secret_access_key)
  documents = []
  for document in s3_resource.Bucket(s3_bucket).objects.all():
    documents.append(document.key)
  return documents

In [0]:
def read_document(document):
  """
  Read Document and return its file content
  :param document: document filename found in s3 by discover_document
  """
  import boto3
  s3_resource = boto3.resource('s3',aws_access_key_id=aws_access_key_id,aws_secret_access_key=aws_secret_access_key)
  document_path = s3_resource.Object(s3_bucket,str(document))
  return document_path.get()['Body'].read().decode('cp1252').splitlines()

In [0]:
def map_words():
  """
  This function works as a mapper.
  """
  word_wordID_docID = []
  word_unique_wordID = []
  
  for document_name in discover_document(s3_bucket):
    for document in read_document(document_name):
      for word in document.split():
        if word not in [w for (w, wID) in word_unique_wordID if w == word]:
          word_wordID_docID.append((len(word_unique_wordID),document_name))
          word_unique_wordID.append((word,len(word_unique_wordID)))
        else:
          w_index = [wID for (w, wID) in word_unique_wordID if w == word]
          word_wordID_docID.append((w_index[0],document_name))
  return(word_wordID_docID, word_unique_wordID)

In [0]:
def create_inverted_index():
  """
  This function works as a reducer.
  """
  inverted_index = []
  word_wordID_docID, word_unique_wordID = map_words()
  
  for wwd_wID,wwd_dID in word_wordID_docID:
    temp_dID_list = set()
    if wwd_wID not in [wID for (wID, dID) in inverted_index if wwd_wID == wID]:
      temp_dID_list.add(wwd_dID)
      inverted_index.append((wwd_wID,list(temp_dID_list)))
    else:
      for i_wID,i_dID in inverted_index:
        if wwd_wID == i_wID:
          temp_dID_list = []
          temp_dID_list = i_dID
          temp_dID_list.append(wwd_dID)
          indice = [wID[0] for wID in inverted_index].index(i_wID)
          inverted_index[indice] = ((wwd_wID,list(set(temp_dID_list))))
  return word_unique_wordID, inverted_index

In [0]:
word_unique_wordID1, inverted_index1 = create_inverted_index()

In [0]:
for i in inverted_index1:
  print(i)

(0, ['3', '1', '0', '2'])
(1, ['3', '1', '0', '2'])
(2, ['3', '1', '0', '2'])
(3, ['3', '1', '0', '2'])
(4, ['3', '1', '0', '2'])
(5, ['3', '1', '0', '2'])
(6, ['3', '1', '0', '2'])
(7, ['3', '1', '0', '2'])
(8, ['1', '0', '2'])
(9, ['1', '0', '2'])
(10, ['1', '0', '2'])
(11, ['1', '0', '2'])
(12, ['1', '0', '2'])
(13, ['1', '0', '2'])
(14, ['1', '0', '2'])
(15, ['1', '0', '2'])
(16, ['1', '0', '2'])
(17, ['0', '1', '3', '2'])
(18, ['1', '0', '2'])
(19, ['1', '0', '3'])
(20, ['1', '0', '3'])
(21, ['1', '0', '3'])
(22, ['1', '0', '3'])
(23, ['1', '0', '3'])
(24, ['1', '0', '3'])
(25, ['1', '0', '3'])
(26, ['1', '0', '3'])
(27, ['1', '0', '3'])
(28, ['1', '0', '3'])
(29, ['1', '0', '3'])
(30, ['1', '0', '3'])
(31, ['1', '0', '3'])
(32, ['1', '0', '3'])
(33, ['1', '0', '3'])
(34, ['1', '0', '3'])
(35, ['1', '0', '3'])
(36, ['1', '0', '3'])
(37, ['1', '0', '3'])
(38, ['1', '0', '3'])
(39, ['1', '0', '3'])
(40, ['1', '0', '3'])
(41, ['1', '0', '3'])
(42, ['1', '0', '3'])
(43, ['1', '0', '3'])
(44, ['1', '0', '3'])
(45, ['1', '0', '3'])
(46, ['1', '0', '3'])
(47, ['1', '0', '3'])
(48, ['1', '0', '3'])
(49, ['1', '0', '3'])
(50, ['1', '0', '3'])
(51, ['1'])
(52, ['1'])
(53, ['1'])
(54, ['1'])
(55, ['1'])
(56, ['1'])
(57, ['1'])
(58, ['1'])
(59, ['1'])
(60, ['1'])
(61, ['1'])
(62, ['1'])
(63, ['1'])
(64, ['1'])
(65, ['1'])

In [0]:
for i in word_unique_wordID1:
  print(i)

('Project', 0)
("Gutenberg's", 1)
('Etext', 2)
('of', 3)
("Shakespeare's", 4)
('First', 5)
('Folio/35', 6)
('Plays', 7)
('This', 8)
('is', 9)
('our', 10)
('3rd', 11)
('edition', 12)
('most', 13)
('these', 14)
('plays.', 15)
('See', 16)
('the', 17)
('index.', 18)
('Please', 19)
('take', 20)
('a', 21)
('look', 22)
('at', 23)
('important', 24)
('information', 25)
('in', 26)
('this', 27)
('header.', 28)
('We', 29)
('encourage', 30)
('you', 31)
('to', 32)
('keep', 33)
('file', 34)
('on', 35)
('your', 36)
('own', 37)
('disk,', 38)
('keeping', 39)
('an', 40)
('electronic', 41)
('path', 42)
('open', 43)
('for', 44)
('next', 45)
('readers.', 46)
('Do', 47)
('not', 48)
('remove', 49)
('this.', 50)
('Copyright', 51)
('laws', 52)
('are', 53)
('changing', 54)
('all', 55)
('over', 56)
('world,', 57)
('be', 58)
('sure', 59)
('check', 60)
('copyright', 61)
('country', 62)
('before', 63)
('posting', 64)
('files!!', 65)